In [6]:
# !pip install focal-loss
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models, regularizers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers.legacy import Adam
from focal_loss import BinaryFocalLoss

In [2]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [3]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use linear activation for Hinge loss
])


In [7]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryFocalLoss(gamma=5), metrics=['accuracy'])

In [8]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
87/87 [==============================] - 15s 162ms/step - loss: 0.0191 - accuracy: 0.7296 - val_loss: 0.0157 - val_accuracy: 0.7778
Epoch 2/20
87/87 [==============================] - 14s 160ms/step - loss: 0.0150 - accuracy: 0.7773 - val_loss: 0.0124 - val_accuracy: 0.8438
Epoch 3/20
87/87 [==============================] - 15s 170ms/step - loss: 0.0126 - accuracy: 0.8084 - val_loss: 0.0102 - val_accuracy: 0.8715
Epoch 4/20
87/87 [==============================] - 15s 168ms/step - loss: 0.0110 - accuracy: 0.8612 - val_loss: 0.0108 - val_accuracy: 0.8854
Epoch 5/20
87/87 [==============================] - 15s 177ms/step - loss: 0.0094 - accuracy: 0.8821 - val_loss: 0.0073 - val_accuracy: 0.8854
Epoch 6/20
87/87 [==============================] - 16s 184ms/step - loss: 0.0090 - accuracy: 0.8807 - val_loss: 0.0072 - val_accuracy: 0.9236
Epoch 7/20
87/87 [==============================] - 16s 182ms/step - loss: 0.0075 - accuracy: 0.9017 - val_loss: 0.0099 - val_accuracy: 0.8750

In [9]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

18/18 [==============================] - 3s 150ms/step - loss: 0.0032 - accuracy: 0.9618
Test Loss: 0.003221520222723484, Test Accuracy: 0.9618055820465088
